# 소아마취 pdf 페이지별로 질문 만들기

- 일단 페이지별 질문 1개

In [ ]:
# grouped_pages.json 불러오기
import json

json_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/data_v2/grouped_pages.json'

# 파일 읽어서 변수에 저장
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(data)

{'6': '# 1. 목적\n국내 소아 진정(pediatric sedation) 실태를 반영한 본 소아 진정 가이드라인의 목적은 실제 소아\n진정을 수행하는 의료진이 진단 및 치료 목적의 시술을 받는 소아에게 안전하고 효과적인 진정을\n수행할 수 있게 하기 위함이다.\n# 2. 정의\n# 1) 진정 진료권자\n- ① 실제 소아 진정을 수행하는 의사로서\n\n- ② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의사\n\n- ③ 깊은 진정의 수행은 진정 약물에 대해 잘 알고, 마스크 환기 및 기관내삽관 등 기도 관\n- 리와 소생술에 능숙한 의사에 의해 시행될 것을 권고한다. 예) 마취통증의학과 의사\n\n# 2) 진정 수행자\n- ① 진정 수행 시에 투약 및 진정 중 활력 징후 감시, 회복 관리를 수행하는 의료진\n\n- ② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의료진\n\n- ③ 진정의 깊이가 깊어지면 기도 관리가 가능한 진정 감시자 인력의 추가를 권장한다.\n\n# 3) 진정의 대상\n진정 하에 진단 및 치료 목적의 시술을 받는 소아\n# 4) 진정 단계1,2\n진정은 그 정도에 따라 최소 진정, 중등도 진정, 깊은 진정, 마취에 이르는 일련의 상태를 말\n1\n본 가이드라인은 대한소아마취학회(The Korean Society of Pediatric Anesthesiologists, KSPA)가 주관하여 편찬한 "소아 진\n정 가이드라인 - 한국형 지침" 입니다.', '7': '한다. 시술의 특성 및 환자 상태에 따라 요구되는 진정의 정도를 계획하여야 하고, 진정이 깊어질\n수록 호흡 및 심혈관계 기능일 저하될 수 있음에 유의하여야 한다. 본 가이드라인은 최소 진정부\n터 깊은 진정까지 적용될 수 있다.\n- ① 최소 진정: 불안을 해소하는 정도의 얕은 진정으로, 구두명령에 반응하고, 호흡계나 심혈\n- 관계 기능은 영향 받지 않는다.\n\n- ② 중등도 진정: 약물에 의해 의식이 저하된 상태이나, 구두명령에 목적 

## 질문 - 텍스트 생성

In [10]:
from openai import OpenAI
import json
from tqdm import tqdm
import os
from dotenv import load_dotenv

# 질문 생성과 관련 텍스트 뽑는 프롬프트 
# system_prompt = """
# 너는 '소아 진정 가이드라인'이라는 pdf 문서를 기반으로 **페이지별 질문을 생성하고 질문을 생성할 때 참고한 문장들을 반환해주는 어시스턴트야.**

# 질문을 생성할 때 참고해야 하는 가이드라인과 질문을 생성할 때 참고한 문장들을 반환할 때 지켜야 할 가이드라인을 참고해줘.

# # 질문 생성 가이드라인
# - 질문은 해당 문서 내용을 잘 이해해야만 풀 수 있어야 합니다.
# - 특히 문서에서 너가 판단하기에 중요한 문장 또는 핵심 개념을 중심으로 질문을 만들어주세요.
# - 답이 너무 명백하거나 단순한 사실 확인이 아닌, 개념을 이해했는지 파악할 수 있는 질문으로 만들어주세요.
# - 객관식이 아닌 서술형 질문으로 만들어주세요.
# - 비판적 사고, 비교, 원인-결과 능력을 평가할 수 있는 질문이면 좋습니다.
# - **무조건 주어진 문서를 기반으로 예상 질문을 생성해주세요.**

# # 질문 생성 때 참고한 문장 반환 가이드라인
# - 질문을 생성할 때 근거가 되는 문장들을 전부 그대로 알려줘.
# - 표를 보고 질문을 생성했다면 표 전체를 반환해줘.
# - 사진과 그림을 보고 질문을 생성했다면 사진과 그림 앞뒤 문장들도 같이 반환해줘.

# # 참고할 예시들
# Context: A Wikipedia paragraph about the United States, 
# Question: How many states are in the United States?

# Context: A Wikipedia paragraph about vampire bats, 
# Question: What are the different species of vampire bats?
# """

system_prompt = """
You are an assistant that generates one question per page based on a PDF document titled "Pediatric Sedation Guidelines".  
For each question you generate, you must also return the exact sentences (or tables/figures) you referred to when creating the question.

Please follow the guidelines below for question generation and reference extraction.

# Question Generation Guidelines
- The question must require a solid understanding of the given page content to answer.
- Focus on important sentences or core concepts from the document when creating the question.
- Avoid simple factual or yes/no questions. Prefer conceptual questions that assess comprehension.
- Use **open-ended (descriptive) questions**, not multiple-choice.
- Questions that encourage critical thinking, comparison, or cause-effect reasoning are strongly preferred.
- **You must generate the question based solely on the given document content. Do not use external knowledge.**

# Reference Extraction Guidelines
- Provide the exact sentences that served as the basis for the question.
- If a table was referenced, include the **entire table**.
- If an image or figure was referenced, include the **sentences immediately before and after** the image as well.

# Output Format
Return the result in the following JSON format:
{
  "question": "<your generated question here>",
  "reference_sentences": [
    "<sentence 1>",
    "<sentence 2>",
    ...
  ]
}

# Example
Context: A Wikipedia paragraph about the United States  
Output:
{
  "question": "How many states are in the United States?",
  "reference_sentences": [
    "The United States is composed of 50 states.",
    "Each state has its own government and constitution."
  ]
}

Context: A Wikipedia paragraph about vampire bats  
Output:
{
  "question": "What are the different species of vampire bats?",
  "reference_sentences": [
    "There are three species of vampire bats: the common vampire bat, the hairy-legged vampire bat, and the white-winged vampire bat."
  ]
}
"""

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 결과 저장용 딕셔너리
results = {}

# 페이지별 질문 생성
for page_number, context in tqdm(data.items()):
    
    user_prompt = f"Context:\n{context}"

    try:
        response = client.responses.create(
            model="gpt-4o-mini",  # 또는 "gpt-3.5-turbo"
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5
        )
        output = response.output[0].content[0].text
        parsed = json.loads(output)
        results[page_number] = parsed

    except Exception as e:
        print(f"Error on page {page_number}: {e}")

# 결과 출력
print(json.dumps(results, indent=2, ensure_ascii=False))

100%|██████████| 58/58 [05:28<00:00,  5.66s/it]

{
  "6": {
    "question": "What are the key roles and qualifications required for medical personnel involved in pediatric sedation according to the guidelines?",
    "reference_sentences": [
      "진정 진료권자 - ① 실제 소아 진정을 수행하는 의사로서",
      "② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의사",
      "③ 깊은 진정의 수행은 진정 약물에 대해 잘 알고, 마스크 환기 및 기관내삽관 등 기도 관리와 소생술에 능숙한 의사에 의해 시행될 것을 권고한다.",
      "진정 수행자 - ① 진정 수행 시에 투약 및 진정 중 활력 징후 감시, 회복 관리를 수행하는 의료진",
      "② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의료진",
      "③ 진정의 깊이가 깊어지면 기도 관리가 가능한 진정 감시자 인력의 추가를 권장한다."
    ]
  },
  "7": {
    "question": "What are the different levels of sedation defined in the guidelines, and how do they impact respiratory and cardiovascular functions?",
    "reference_sentences": [
      "본 가이드라인은 최소 진정부터 깊은 진정까지 적용될 수 있다.",
      "① 최소 진정: 불안을 해소하는 정도의 얕은 진정으로, 구두명령에 반응하고, 호흡계나 심혈관계 기능은 영향 받지 않는다.",
      "② 중등도 진정: 약물에 의해 의식이 저하된 상태이나, 구두명령에 목적 있는 반응을 보인다.",
      "대개 자발적인 환기가 유지되며, 심혈관계 기능도 대개 유지된다.",
      "③ 깊은 진정: 약물에 의한 수면상태로

In [12]:
# JSON 파일에 저장 (UTF-8 인코딩 및 한글 출력 보장)
with open('question_context.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("저장완.")

저장완.


In [ ]:
json_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/grouped_pages.json'

# 파일 읽어서 변수에 저장
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(data)

## 답변 생성

In [10]:
import json

json_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/question_context.json'

# 파일 읽어서 변수에 저장
with open(json_file_path, 'r', encoding='utf-8') as f:
    question_context = json.load(f)

print(question_context)

{'6': {'question': 'What are the key roles and qualifications required for medical personnel involved in pediatric sedation according to the guidelines?', 'reference_sentences': ['진정 진료권자 - ① 실제 소아 진정을 수행하는 의사로서', '② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의사', '③ 깊은 진정의 수행은 진정 약물에 대해 잘 알고, 마스크 환기 및 기관내삽관 등 기도 관리와 소생술에 능숙한 의사에 의해 시행될 것을 권고한다.', '진정 수행자 - ① 진정 수행 시에 투약 및 진정 중 활력 징후 감시, 회복 관리를 수행하는 의료진', '② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의료진', '③ 진정의 깊이가 깊어지면 기도 관리가 가능한 진정 감시자 인력의 추가를 권장한다.']}, '7': {'question': 'What are the different levels of sedation defined in the guidelines, and how do they impact respiratory and cardiovascular functions?', 'reference_sentences': ['본 가이드라인은 최소 진정부터 깊은 진정까지 적용될 수 있다.', '① 최소 진정: 불안을 해소하는 정도의 얕은 진정으로, 구두명령에 반응하고, 호흡계나 심혈관계 기능은 영향 받지 않는다.', '② 중등도 진정: 약물에 의해 의식이 저하된 상태이나, 구두명령에 목적 있는 반응을 보인다.', '대개 자발적인 환기가 유지되며, 심혈관계 기능도 대개 유지된다.', '③ 깊은 진정: 약물에 의한 수면상태로 반복적인 아픈 자극에는 목적 있는 반응을 보일 수 있다.', '자발호흡이 부적절할 수 있고, 때때로 기도 유지가 필요하지만, 심혈관계 기능은 대개 잘 유지된다.', '④ 전신 

In [21]:
for i,j in question_context.items():
    print(i)

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [17]:
from openai import OpenAI
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

answer_prompt = """
You are given a question and a context. Use the information in the context to answer the question carefully.

Instructions:
- Please answer in Korean.
- Begin by reasoning step-by-step how the context leads to the answer.
- If you quote directly from the context, enclose the quote within ##begin_quote## and ##end_quote##.
- Your reasoning should combine your own interpretation and these quoted parts.
- Conclude your response with a final answer in the format: <ANSWER>: $answer
- The answer must be succinct and begin with the tag "<ANSWER>:"
- Do not add your own knowledge or assumptions. Your answer must be solely based on the provided context.
- When citing the source from the context, enclose the quoted sentence using ##begin_quote## and ##end_quote##.

# Example:
Question: The Oberoi family is part of a hotel company that has a head office in what city?

Context: [The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group...It is located in city center of Jakarta, near Mega Kuningan, adjacent to the sister JW Marriott Hotel. It is operated by The Ritz-Carlton Hotel Company. The complex has two towers that comprises a hotel and the Airlangga Apartment respectively...The Oberoi Group is a hotel company with its head office in Delhi.]

CoT Answer:
##Reason: The document ##begin_quote## The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group. ##end_quote## establishes that the Oberoi family is involved in the Oberoi Group, and the document ##begin_quote## The Oberoi Group is a hotel company with its head office in Delhi. ##end_quote## establishes the head office of The Oberoi Group. Therefore, the Oberoi family is part of a hotel company whose head office is in Delhi. 
<ANSWER>: Delhi

Now apply this to the question and context below.
"""

results = {}

for i,content in tqdm(question_context.items()):
    question = content['question']
    context = content['reference_sentences']

    user_prompt = f"Question:\n{question}, Context:\n{context}"
    
    try:
        response = client.responses.create(
            model="gpt-4o-mini", 
            input=[
                {"role": "system", "content": answer_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5
        )
        output = response.output[0].content[0].text
        results[i] = output

    except Exception as e:
        print(f"Error on page {i}: {e}")

# 결과 출력
print(json.dumps(results, indent=2, ensure_ascii=False))

100%|██████████| 58/58 [06:40<00:00,  6.90s/it]

{
  "6": "##Reason: 문서에서 \"진정 진료권자\"의 역할은 실제 소아 진정을 수행하는 의사로 명시되어 있으며, 이들은 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수해야 합니다. 또한, 깊은 진정을 수행하기 위해서는 진정 약물에 대한 지식과 기도 관리 및 소생술에 능숙해야 한다고 합니다. 이는 ##begin_quote## \"① 실제 소아 진정을 수행하는 의사로서\", \"② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의사\", \"③ 깊은 진정의 수행은 진정 약물에 대해 잘 알고, 마스크 환기 및 기관내삽관 등 기도 관리와 소생술에 능숙한 의사에 의해 시행될 것을 권고한다.\" ##end_quote##에서 확인할 수 있습니다. \n\n또한, \"진정 수행자\"의 역할은 진정 중 활력 징후 감시와 회복 관리 등을 수행하는 의료진으로, 이들도 진정 교육과 심혈관계 중재 및 기도 유지 교육을 이수해야 하며, 깊은 진정 시에는 기도 관리가 가능한 추가 인력이 권장된다고 합니다. 이는 ##begin_quote## \"① 진정 수행 시에 투약 및 진정 중 활력 징후 감시, 회복 관리를 수행하는 의료진\", \"② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의료진\", \"③ 진정의 깊이가 깊어지면 기도 관리가 가능한 진정 감시자 인력의 추가를 권장한다.\" ##end_quote##에서 확인할 수 있습니다.\n\n따라서, 소아 진정에 관여하는 의료 인력은 진정 교육과 심혈관계 중재 및 기도 유지 교육을 이수해야 하며, 기도 관리와 소생술에 능숙해야 하는 것이 중요합니다. \n<ANSWER>: 진정 교육과 심혈관계 중재 및 기도 유지 교육 이수, 기도 관리 및 소생술 능숙.",
  "7": "##Reason: 주어진 문서에서는 진정의 네 가지 수준이 정의되어 있습니다. 첫째, 최소 진정은 불안을 해소하는 얕은 진정으로, 구두명령에 반응하며 호흡계나 심혈관계 기능에 영향을 미치지 않는다고 합니다(##begi

In [18]:
# JSON 파일에 저장 (UTF-8 인코딩 및 한글 출력 보장)
with open('answer.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("저장완.")

저장완.


## csv로 합치기

In [19]:
json_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/answer.json'

# 파일 읽어서 변수에 저장
with open(json_file_path, 'r', encoding='utf-8') as f:
    answer = json.load(f)

print(answer)

{'6': '##Reason: 문서에서 "진정 진료권자"의 역할은 실제 소아 진정을 수행하는 의사로 명시되어 있으며, 이들은 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수해야 합니다. 또한, 깊은 진정을 수행하기 위해서는 진정 약물에 대한 지식과 기도 관리 및 소생술에 능숙해야 한다고 합니다. 이는 ##begin_quote## "① 실제 소아 진정을 수행하는 의사로서", "② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의사", "③ 깊은 진정의 수행은 진정 약물에 대해 잘 알고, 마스크 환기 및 기관내삽관 등 기도 관리와 소생술에 능숙한 의사에 의해 시행될 것을 권고한다." ##end_quote##에서 확인할 수 있습니다. \n\n또한, "진정 수행자"의 역할은 진정 중 활력 징후 감시와 회복 관리 등을 수행하는 의료진으로, 이들도 진정 교육과 심혈관계 중재 및 기도 유지 교육을 이수해야 하며, 깊은 진정 시에는 기도 관리가 가능한 추가 인력이 권장된다고 합니다. 이는 ##begin_quote## "① 진정 수행 시에 투약 및 진정 중 활력 징후 감시, 회복 관리를 수행하는 의료진", "② 진정 교육과 심혈관계 중재 및 기도 유지에 관한 교육을 이수한 의료진", "③ 진정의 깊이가 깊어지면 기도 관리가 가능한 진정 감시자 인력의 추가를 권장한다." ##end_quote##에서 확인할 수 있습니다.\n\n따라서, 소아 진정에 관여하는 의료 인력은 진정 교육과 심혈관계 중재 및 기도 유지 교육을 이수해야 하며, 기도 관리와 소생술에 능숙해야 하는 것이 중요합니다. \n<ANSWER>: 진정 교육과 심혈관계 중재 및 기도 유지 교육 이수, 기도 관리 및 소생술 능숙.', '7': "##Reason: 주어진 문서에서는 진정의 네 가지 수준이 정의되어 있습니다. 첫째, 최소 진정은 불안을 해소하는 얕은 진정으로, 구두명령에 반응하며 호흡계나 심혈관계 기능에 영향을 미치지 않는다고 합니다(##begin_quote## '구두명령에 반응하고

In [24]:
answer.get(6, "")

''

In [25]:
import pandas as pd

rows = []

for i,j in question_context.items():

    question = j['question']
    context = j['reference_sentences']

    answer_ = answer.get(i, "")

    rows.append({
            'question': question,
            'context': context,
            'answer': answer_})
    
df = pd.DataFrame(rows)
print(df)
df.to_csv("output.csv", index=False, encoding="utf-8")

                                             question  \
0   What are the key roles and qualifications requ...   
1   What are the different levels of sedation defi...   
2   What are the critical components involved in t...   
3   What key components should be considered durin...   
4   What are the qualifications and responsibiliti...   
5   What considerations must be taken into account...   
6   How should the frequency of monitoring vital s...   
7   What monitoring protocols are recommended for ...   
8   What are the key criteria for patient discharg...   
9   What are the key considerations and steps invo...   
10  How does the ASA Physical Status Classificatio...   
11  What factors should be considered when assessi...   
12  What considerations should be taken into accou...   
13  How do the fasting guidelines differ for child...   
14  What are the essential components that must be...   
15  Discuss the advantages and potential complicat...   
16  What are the critical monit